In [ ]:
# Install necessary packages
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 google-cloud-pipeline-components \
                                 kfp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 41.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which

In [ ]:
# Set project ID
PROJECT_ID = "***"

# Set the project id
! gcloud config set project {PROJECT_ID} --quiet

Updated property [core/project].


In [ ]:
REGION = "us-central1"

In [ ]:
# Authenticate with Google Cloud
! gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=1blrQ2KCpscC1hzx2CZB6isBHhxPKa&prompt=consent&access_type=offline&code_challenge=opY-gN-Xlp4HtnEo9v9HTfh8bfqcW5yBThbLPRpXvWs&code_challenge_method=S256

Enter authorization code: 4/0AeaYSHAYC6aZAUUCsSyzOz-nklJ27S_TDgskEt4WOKwQ2yjs6PNMscMpDG--2NO2w2Lsqg

You are now logged in as [rashadha.rimsan@ncinga.net].
Your current project is [data-science-projects-393906].  You can change this setting by running:
  $ gcloud conf

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Create a bucket URI
BUCKET_URI = f"gs://bidirectional-lstm-model-{PROJECT_ID}-unique"

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://bidirectional-lstm-model-data-science-projects-393906-unique/...
ServiceException: 409 A Cloud Storage bucket named 'bidirectional-lstm-model-data-science-projects-393906-unique' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [ ]:
# Set service account
SERVICE_ACCOUNT = "***"

In [ ]:
import sys

# Use default service account if not provided
IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "***"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

Service Account: 777232604101-compute@developer.gserviceaccount.com


In [ ]:
# Set IAM permissions for the service account
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://bidirectional-lstm-model-data-science-projects-393906-unique/
No changes made to gs://bidirectional-lstm-model-data-science-projects-393906-unique/


In [ ]:
# Import necessary libraries
import os
from typing import Any, Dict, List

import google.cloud.aiplatform as aip
import kfp
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components.v1.custom_job.component import \
    custom_training_job as CustomTrainingJobOp
from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                          ModelDeployOp)
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from kfp import compiler
from kfp.dsl import importer_node

In [ ]:
# Set pipeline root to save all model params
PIPELINE_ROOT = "{}/pipeline_root/tpu_bidirectional_pipeline".format(BUCKET_URI)

In [ ]:
# Initialize Vertex AI with staging bucket to store models artifacts
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [ ]:
# provides access to Google Cloud's AI Platform services
from google.cloud.aiplatform import gapic

# Define accelerator types for training and deployment
# assigned values for the accelerator type and the number of accelerators to use during training
TRAIN_TPU, TRAIN_NTPU = (
    gapic.AcceleratorType.TPU_V2,
    8,
)  # Using TPU_V2 with 8 accelerators

#  assigning values for the accelerator type and the number of accelerators to use during deployment
DEPLOY_GPU, DEPLOY_NGPU = (gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

In [ ]:
# Define deployment details
DEPLOY_VERSION = "tf2-gpu.2-9"

# creating a deployment image URI based on the deployment version
DEPLOY_IMAGE = "us-docker.pkg.dev/cloud-aiplatform/prediction/{}:latest".format(
    DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

Deployment: us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-gpu.2-9:latest AcceleratorType.NVIDIA_TESLA_K80 1


In [ ]:
# Define compute types for training and deployment
# define the machine type for training
MACHINE_TYPE = "cloud-tpu"

# TPU VMs do not require VCPU definition

# the training will be done on TPU VMs
TRAIN_COMPUTE = MACHINE_TYPE
print("Train machine type", TRAIN_COMPUTE)

# define the machine type for deployment
MACHINE_TYPE = "n1-standard"

VCPU = "4" #  using in the construction of the deployment compute type
# the deployment will be on a machine with 4 virtual CPUs
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

Train machine type cloud-tpu
Deploy machine type n1-standard-4


In [ ]:
# Determine training strategy based on number of TPUs

# if TRAIN_NTPU is not specified or if it's less than 2
if not TRAIN_NTPU or TRAIN_NTPU < 2:
    # GPU will be used for training
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "tpu"

EPOCHS = 20
STEPS = 10000

# Define trainer arguments
TRAINER_ARGS = [
    "--epochs=" + str(EPOCHS),
    "--steps=" + str(STEPS),
    "--distribute=" + TRAIN_STRATEGY,
    "--bucket_name=" + "a-tpu-training",
    "--train_data=" + "BTCUSDT_ROOLING_1H-000000000001.csv"
]

# create working dir to pass to job spec
WORKING_DIR = f"{PIPELINE_ROOT}/model"

MODEL_DISPLAY_NAME = "tpu_train_deploy"
print(TRAINER_ARGS, WORKING_DIR, MODEL_DISPLAY_NAME)

['--epochs=20', '--steps=10000', '--distribute=tpu', '--bucket_name=a-tpu-training', '--train_data=BTCUSDT_ROOLING_1H-000000000001.csv'] gs://bidirectional-lstm-model-data-science-projects-393906-unique/pipeline_root/tpu_bidirectional_pipeline/model tpu_train_deploy


In [ ]:
%%writefile Dockerfile
# Specifies base image and tag
FROM us-docker.pkg.dev/vertex-ai/training/tf-tpu-pod-base-cp38:latest
WORKDIR /root

# Download and install `tensorflow`.
RUN pip install https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/tensorflow/tf-2.12.0/tensorflow-2.12.0-cp38-cp38-linux_x86_64.whl

RUN pip3 install gcsfs google-cloud-storage scikit-learn pandas numpy argparse

# Download and install `libtpu`.
# save `libtpu.so` in the '/lib' directory of the container image.
# libtpu is a library for interfacing with TPUs
RUN curl -L https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/libtpu/1.6.0/libtpu.so -o /lib/libtpu.so

# Copies the trainer code to the docker image.
COPY train.py /root/train.py

ENTRYPOINT ["python3", "train.py"]

Overwriting Dockerfile


In [ ]:
%%writefile train.py
# Single, Mirror and Multi-Machine Distributed Training for tpu_bidirectional_pipeline

import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, LayerNormalization
from tensorflow.keras.optimizers import Adam

parser = argparse.ArgumentParser()
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--bucket_name', type=str, required=True,
                    help='Bucket name on GCS')
parser.add_argument('--train_data', type=str, required=True,
                    help='Path to training data in GCS bucket')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        #  training will be distributed across a single GPU
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        # training will be done on the CPU
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple TPU devices
elif args.distribute == 'tpu':
    # Connects to the TPU cluster and initializes the TPU system
    cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")
    tf.config.experimental_connect_to_cluster(cluster_resolver)
    tf.tpu.experimental.initialize_tpu_system(cluster_resolver)

    # distribute training across multiple TPUs in the cluster
    strategy = tf.distribute.TPUStrategy(cluster_resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    # performs synchronous training across multiple GPUs on a single machine
    # Each GPU will process a different batch of data
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    # performs distributed synchronous training across multiple workers
    # Each with potentially multiple GPUs
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


# Function to read data from GCS
def read_data_from_gcs(bucket_name, train_data_path):
    # Construct GCS path for the training data
    gcs_input_path = f"gs://{bucket_name}/{train_data_path}"
    # Read training data from GCS
    return pd.read_csv(gcs_input_path)


# Preparing dataset
# Function to prepare the dataset
def prepare_dataset(feature_array, sequence_length, sampling_rate, batch_size, target_scaler):
    last = sequence_length * sampling_rate + sampling_rate
    targets_normalized = target_scaler.fit_transform(feature_array[:-last][:, 6].reshape(-1, 1))
    dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=feature_array[sampling_rate:-sequence_length * sampling_rate],
        targets=targets_normalized,
        sequence_length=sequence_length,
        sequence_stride=1,
        sampling_rate=sampling_rate,
        shuffle=False,
        batch_size=batch_size,
    )
    return dataset

# Build the Keras model
def build_and_compile_model():
    # Model definition
    model = Sequential([
        LayerNormalization(axis=-1),  # First layer of the model
        Bidirectional(LSTM(50, return_sequences=True), input_shape=(50, 12)),
        Dropout(0.2),
        Bidirectional(LSTM(50, return_sequences=True)),
        Dropout(0.2),
        Bidirectional(LSTM(50, return_sequences=True)),
        Dropout(0.2),
        Bidirectional(LSTM(25)),
        Dropout(0.2),
        Dense(5),
        Dense(1)
    ])

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='mean_absolute_error')

    return model

# Train the model
MODEL_DIR = os.getenv("AIP_MODEL_DIR")
# Read data
full_df = read_data_from_gcs(args.bucket_name, args.train_data)

# Preprocess data
feature_columns = full_df[["open", "high", "low", "close", "price_change_sign", "price_change", "price_change_percentage", "volume", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume"]]
feature_array = feature_columns.to_numpy()

# Define target scaler
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Adjust batch_size for optimization
batch_size = 32

# Prepare the dataset
dataset = prepare_dataset(feature_array, 50, 60, batch_size, target_scaler)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_model()

model.fit(dataset, epochs=args.epochs, steps_per_epoch=args.steps)
if args.distribute=="tpu":
    # model will be saved locally on the TPU instance
    save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
    # model is saved to the specified directory
    model.save(MODEL_DIR, options=save_locally)
else:
    model.save(MODEL_DIR)

Writing train.py


In [ ]:
# Enable Artifact Registry service
! gcloud services enable artifactregistry.googleapis.com

# Update Google Cloud SDK components if in testing environment
if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

In [ ]:
# Create Docker group
!sudo groupadd docker

groupadd: group 'docker' already exists


In [ ]:
# Add user to Docker group
!sudo usermod -a -G docker ${USER}

Usage: usermod [options] LOGIN

Options:
  -b, --badnames                allow bad names
  -c, --comment COMMENT         new value of the GECOS field
  -d, --home HOME_DIR           new home directory for the user account
  -e, --expiredate EXPIRE_DATE  set account expiration date to EXPIRE_DATE
  -f, --inactive INACTIVE       set password inactive after expiration
                                to INACTIVE
  -g, --gid GROUP               force use GROUP as new primary group
  -G, --groups GROUPS           new list of supplementary GROUPS
  -a, --append                  append the user to the supplemental GROUPS
                                mentioned by the -G option without removing
                                the user from other groups
  -h, --help                    display this help message and exit
  -l, --login NEW_LOGIN         new value of the login name
  -L, --lock                    lock the user account
  -m, --move-home               move contents of the home direc

In [ ]:
# Define repository and image names
REPOSITORY = "tpu-training-repository"
IMAGE = "tpu-train"

In [ ]:
# Create Artifact Registry repository
!gcloud artifacts repositories create $REPOSITORY --repository-format=docker \
--location=us-central1 --description="Vertex TPU training repository"

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [ ]:
# Define train image
TRAIN_IMAGE = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}:latest"

In [ ]:
print(TRAIN_IMAGE)

us-central1-docker.pkg.dev/data-science-projects-393906/tpu-training-repository/tpu-train:latest


In [ ]:
# Submit build to Google Cloud Build
! gcloud builds submit --region=us-central1 --tag $TRAIN_IMAGE

Creating temporary tarball archive of 63 file(s) totalling 57.4 MiB before compression.
Uploading tarball of [.] to [gs://data-science-projects-393906_cloudbuild/source/1708017328.172113-726b8bc959e140c3bef4a0476f0732a1.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/data-science-projects-393906/locations/us-central1/builds/ae7353e8-e5f9-4bb8-993c-a92a94c78b5d].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=us-central1/ae7353e8-e5f9-4bb8-993c-a92a94c78b5d?project=777232604101 ].
 REMOTE BUILD OUTPUT
starting build "ae7353e8-e5f9-4bb8-993c-a92a94c78b5d"

FETCHSOURCE
Fetching storage object: gs://data-science-projects-393906_cloudbuild/source/1708017328.172113-726b8bc959e140c3bef4a0476f0732a1.tgz#1708017338642719
Copying gs://data-science-projects-393906_cloudbuild/source/1708017328.172113-726b8bc959e140c3bef4a0476f0732a1.tgz#1708017338642719...
/ [1 files][  6.8 MiB/  6.8 MiB]                                                
Operation comp

In [ ]:
! pip install functions-framework==3.*

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
import functions_framework
from google.cloud import pubsub

# Initialize Google Cloud Pub/Sub client
pubsub_client = pubsub.SubscriberClient()

# Define the subscription name
subscription_name = "projects/data-science-projects-393906/subscriptions/changeInStorageBucket-sub"

# Define a global variable to store the index
index_value = None

@functions_framework.cloud_event
def extract_index(cloud_event):
    index_value = None

    # Define the function to process Pub/Sub messages
    def process_message(message):
        nonlocal index_value
        # Extract the index from the Pub/Sub message
        index = int(message.data.decode("utf-8"))

        # Save the index value
        index_value = index

        # Acknowledge the message
        message.ack()

    # Subscribe to the Pub/Sub topic
    subscriber = pubsub_client.subscribe(subscription_name, callback=process_message)

    # Wait for messages
    subscriber.result()  # This will wait indefinitely for new messages

    # Return True if index is not None
    return index_value, index_value is not None


In [ ]:
# Call the function
result, pub_sub_condition = extract_index(None)

In [ ]:
from google.cloud import storage
from io import BytesIO

def check_data_skewness(gcs_bucket, file_path):
    # Initialize GCS client
    client = storage.Client()

    # Get bucket and blob
    bucket = client.get_bucket(gcs_bucket)
    blob = bucket.blob(file_path)

    # Download CSV file from GCS
    csv_data = blob.download_as_string()

    # Read CSV data into pandas DataFrame
    df = pd.read_csv(BytesIO(csv_data))

    # Check if 'price_change_percentage' column exists
    if 'price_change_percentage' not in df.columns:
        return "Error: 'price_change_percentage' column not found in the dataset."

    # Calculate skewness of 'price_change_percentage' column
    skewness = df['price_change_percentage'].skew()

    # Interpret skewness
    if -0.5 <= skewness <= 0.5:
        interpretation = "Fairly symmetrical distribution"
        return False
    elif -1 <= skewness < -0.5 or 0.5 < skewness <= 1:
        interpretation = "Moderately skewed distribution"
        return False
    elif skewness < -1 or skewness > 1:
        interpretation = "Highly skewed distribution"
        return True

In [ ]:
import numpy as np
def calculate_drift_threshold(gcs_bucket, file_index):
    """
    Calculate the drift threshold based on the standard deviation of 'price_change_percentage' values
    in the CSV file.

    Parameters:
        gcs_bucket (str): Name of the Google Cloud Storage bucket.
        file_index (int): Index to split the old and new data.

    Returns:
        float: Drift threshold.
    """
    # Initialize GCS client
    client = storage.Client()

    # Get bucket and blob
    bucket = client.get_bucket(gcs_bucket)
    blob = bucket.blob(file_index)

    # Download CSV file from GCS
    csv_data = blob.download_as_string()

    # Read CSV data into pandas DataFrame
    df = pd.read_csv(BytesIO(csv_data))

    # Get old data up to the specified index
    old_data = df.loc[:file_index, 'price_change_percentage']

    # Calculate standard deviation of old data
    std_dev = np.std(old_data)

    # Define drift threshold as a multiplier of the standard deviation
    multiplier = 2
    threshold = multiplier * std_dev

    return threshold


In [ ]:
def load_model_from_gcs(gcs_path):
    """
    Load a TensorFlow SavedModel from Google Cloud Storage.

    Parameters:
        gcs_path (str): The path to the SavedModel directory in Google Cloud Storage.

    Returns:
        tf.saved_model.Saveable: The loaded TensorFlow SavedModel.
    """
    # Load the model from GCS
    model = tf.saved_model.load(gcs_path)

    return model

In [ ]:
gcs_model_path = "gs://bidirectional-lstm-model-data-science-projects-393906-model/pipeline_root/tpu_bidirectional_pipeline/model"
loaded_model = load_model_from_gcs(gcs_model_path)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
def calculate_prediction_drift(gcs_bucket, file_index, model,drift_threshold):
    # Initialize GCS client
    client = storage.Client()

    # Get bucket and blob
    bucket = client.get_bucket(gcs_bucket)
    blob = bucket.blob(file_index)

    # Download CSV file from GCS
    csv_data = blob.download_as_string()

    # Read CSV data into pandas DataFrame
    df = pd.read_csv(BytesIO(csv_data))

    # Check if 'price_change_percentage' column exists
    if 'price_change_percentage' not in df.columns:
        return "Error: 'price_change_percentage' column not found in the dataset."

    # Assuming 'price_change_percentage' is a feature used by the model
    old_data = df.loc[:file_index, 'price_change_percentage']
    new_data = df.loc[file_index:, 'price_change_percentage']

    # Make predictions on new data
    new_predictions = model.predict(new_data)

    # Make predictions on old data
    original_predictions = model.predict(old_data)

    # Calculate prediction drift (e.g., mean absolute error)
    prediction_drift = mean_absolute_error(original_predictions, new_predictions)

    # Optionally, you can set a threshold to determine if the drift is significant

    # Determine if prediction drift is significant
    if prediction_drift > drift_threshold:
        return True
    else:
        return False

In [ ]:
WORKER_POOL_SPECS = [
    {
        "containerSpec": {
            "args": TRAINER_ARGS,
            "env": [{"name": "AIP_MODEL_DIR", "value": WORKING_DIR}],
            "imageUri": TRAIN_IMAGE,
        },
        "replicaCount": "1",
        "machineSpec": {
            "machineType": TRAIN_COMPUTE,
            "accelerator_type": TRAIN_TPU,
            "accelerator_count": TRAIN_NTPU,
        },
    }
]

In [ ]:
def deploy_model_to_endpoint(
    project: str,
    model_display_name: str,
    serving_container_image_uri: str,
    import_unmanaged_model_task,
    WORKER_POOL_SPECS,
    DEPLOY_COMPUTE,
    DEPLOY_NGPU,
    DEPLOY_GPU,
):
    # training job
    custom_job_task = CustomTrainingJobOp(
        display_name="tpu model training",
        worker_pool_specs=WORKER_POOL_SPECS,
    )

    # uploads the trained model to Google Cloud AI Platform
    model_upload_op = ModelUploadOp(
        project=project,
        display_name=model_display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    )

    # creates an endpoint in Google Cloud AI Platform for serving the model
    endpoint_create_op = EndpointCreateOp(
        project=project,
        display_name="tpu-pipeline-created-endpoint",
    )

    # deploys the uploaded model to the created endpoint
    _ = ModelDeployOp(
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        # deployment configuration
        deployed_model_display_name=model_display_name,
        dedicated_resources_machine_type=DEPLOY_COMPUTE,
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
        dedicated_resources_accelerator_type=DEPLOY_GPU.name,
        dedicated_resources_accelerator_count=DEPLOY_NGPU,
    )

In [ ]:
import kfp.dsl as dsl
from kfp.v2.dsl import component, Input, Output
from google.cloud import pubsub

# Initialize Google Cloud Pub/Sub client
pubsub_client = pubsub.PublisherClient()

# Define Pub/Sub topic name
send_ct_pubsub_topic = "projects/data-science-projects-393906/topics/publishMesssageCT"

@kfp.dsl.pipeline(name="train-endpoint-deploy")
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    serving_container_image_uri: str = DEPLOY_IMAGE,
    gcs_bucket: str = GCS_BUCKET,
    file_index: int = FILE_INDEX,
    drift_threshold: float = DRIFT_THRESHOLD,
    pub_sub_condition: bool = pub_sub_condition,
    file_index: int = index,
    file_path: str = "CPU-23478.csv",
    loaded_model:
):
    # Define a condition to check for data skewness and prediction drift
    with dsl.Condition(pub_sub_condition == True):
        # Check data skewness
        check_skewness = check_data_skewness(gcs_bucket, file_path)

        # Check if data skewness is significant
        with dsl.Condition(check_skewness == True):
            # Publish a Pub/Sub message indicating significant data skewness
            pubsub_message = "Significant data skewness detected."
            pubsub_client.publish(send_ct_pubsub_topic, data=pubsub_message.encode("utf-8"))

            # Start continuous training pipeline here
            # Deployment operations
                deploy_model_to_endpoint(
                    project,
                    model_display_name,
                    serving_container_image_uri,
                    import_unmanaged_model_task,
                    WORKER_POOL_SPECS,
                    DEPLOY_COMPUTE,
                    DEPLOY_NGPU,
                    DEPLOY_GPU,
                )

        # Check if data skewness is not significant
        with dsl.Condition(check_skewness == False):
            drift_threshold = calculate_drift_threshold(gcs_bucket, file_index)
            drift_significant = calculate_prediction_drift(gcs_bucket, file_index, loaded_model, drift_threshold)

            # Publish a Pub/Sub message if prediction drift is significant
            with dsl.Condition(drift_significant == True):
                # Publish a message to Pub/Sub topic
                pubsub_message = "Significant prediction drift detected. Starting continuous training."
                pubsub_client.publish(send_ct_pubsub_topic, data=pubsub_message.encode("utf-8"))

                # Deployment operations
                deploy_model_to_endpoint(
                    project,
                    model_display_name,
                    serving_container_image_uri,
                    import_unmanaged_model_task,
                    WORKER_POOL_SPECS,
                    DEPLOY_COMPUTE,
                    DEPLOY_NGPU,
                    DEPLOY_GPU,
                )


In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="tpu_train_bidirectional_pipeline.json",
)

In [ ]:
DISPLAY_NAME = "tpu_bidirectional_training"

# Creating Pipeline Job
job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="tpu_train_bidirectional_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
)

job.run()

# After the job is executed, the JSON template file
! rm tpu_train_bidirectional_pipeline.json

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/777232604101/locations/us-central1/pipelineJobs/train-endpoint-deploy-20240215171853
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/777232604101/locations/us-central1/pipelineJobs/train-endpoint-deploy-20240215171853')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/train-endpoint-deploy-20240215171853?project=777232604101
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/777232604101/locations/us-central1/pipelineJobs/train-endpoint-deploy-20240215171853 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/777232604101/locations/us-cen

In [ ]:
def get_task_detail(
    task_details: List[Dict[str, Any]], task_name: str
) -> List[Dict[str, Any]]:
    for task_detail in task_details:
      if task_detail.task_name == task_name:
          return task_detail

In [ ]:
DISPLAY_NAME = 'tpu_bidirectional_training'

# Creating Pipeline Job
job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path='tpu_train_bidirectional_pipeline.json'
    pipeline_root = PIPELINE_ROOT,
)

In [ ]:
# Warning: Setting this to true will delete everything in the bucket
delete_bucket = False

if delete_bucket and 'BUCKET_URI' in globals():
  ! gsutil rm -r $BUCKET_URI

In [ ]:
from kfp.v2 import dsl
from kfp.v2.dsl import component, Output

# Define Cleanup pipeline
@dsl.pipeline(name='cleanup-pipeline')
def cleanup_pipeline(
    project: str = PROJECT_ID,
    endpoint_name: str = 'tpu-pipeline-created-endpoint',
    model_display_name: str = MODEL_DISPLAY_NAME
):
    # Delete deployed model
    delete_model_op = ModelDeleteOp(
        project=project,
        model_display_name= model_display_name,
        delete_contents=True, # Optional: Delete Model Contents
    )

    # Delete endpoint
    delete_endpoint_op = EndpointDleleteOp(
        project=project,
        endpoint_name=endpoint_name,
    )

    # Optional: Define pipeline outputs
    # output relevant information or success indicators
    cleanup_completed = Output(bool, description='Cleanup completed successfully.')

In [ ]:
# Define pipeline
@kfp.dsl.pipeline(name='train-endpoint-deploy')
def pipeline(
    project: str = PROJECT_ID,
    model_deploy_name: str = MODEL_DISPLAY_NAME,
    serving_container_image_uri: str = DEPLOY_IMAGE
):

    custom_job_task = CustomTrainingJobOp(
        display_name='tpu model training',
        woker_pool_specs=WORKER_POOL_SPECS,
    )

    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=WORKING_DIR,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            'containerSpec': {
                'imageUri': serving_container_image_uri
            },
        },
    ).after(custome_job_task)

    model_upload_op = ModelUploadOp(
        project=project,
        display_name = model_display_name,
        unmanaged_container_model=import_unmanaged_model_task_outputs['artifacts'],
    )

    endpoint_create_op = EndpointCreateOp(
        project=project,
        display_name='tpu-pipeline-created-endpoint'
    )

    _ = ModelDeployOp(
        endpoint = endpoint_create_op.outputs['endpoint'],
        model = model_upload_op.outputs['model'],
        deployed_model_display_name=model_display_name,
        dedicated_resources_machine_type=DEPLOY_COMPUTE,
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
        dedicated_resources_accelerator_type+DEPLOY_GPU.name,
        dedicated_resources_accelerator_aount=DEPLOY_NGPU,
    )

In [ ]:
kubect1 apply -k 'github.com/kubeflow/katib.git/manifest/'